We consider a system modeled as follows:

$
\begin{cases}
\delta Y_i = \kappa_i D_i\\
X_i = \alpha_i c_i D_i\\
c_i = \beta_i d u_i
\end{cases}
$

with

$
\begin{cases}
d_t = d + \sum c_i + \sum X_i\\
D_{i,t} = D_i + X_i
\end{cases}
$

Our objective is to guarantee that $Y_i \le Y_i^L$ when $u_i \ge u_i^L$.

### Analysis

We reduce the number of equations by noting that

$D_{i,t} = D_i + X_i = D_i (1 + \alpha_i c_i) \Leftrightarrow D_i = D_{i,t} \frac{1}{1 + \alpha_i c_i}$

We can thus write $X_i = D_{i,t} \frac{\alpha_i c_i}{1 + \alpha_i c_i}$, which means that

$
d_t = d + \sum \left( c_i + D_{i,t} \frac{\alpha_i c_i}{1 + \alpha_i c_i} \right)
$

Therefore, we have

$
\frac{c_i / \beta_i}{d_t - \sum \left( c_i + D_{i,t} \frac{\alpha_i c_i}{1 + \alpha_i c_i} \right)} = u_i
$

And the top-level requirement is

$Y_i^L \ge Y_i = \kappa_i D_{i,t} \frac{1}{1 + \alpha_i c_i} \Leftrightarrow \alpha_i c_i \ge \frac{\kappa_i D_{i,t}}{\delta Y_i^L} - 1$

We introduce the new variables $\tilde \beta_i = c_i / \beta_i$, $\tilde \alpha_i = \frac{\alpha_i c_i}{1 + \alpha_i c_i}$, and $\tilde \kappa_i = \frac{\delta Y_i^L}{\kappa_i D_{i,t}}$. We obtain

$
\frac{\tilde \beta_i}{d_t - \sum \left( c_i + D_{i,t} \tilde \alpha_i \right)} = u_i
$

and the top-level requirement is

$
\alpha_i c_i \ge \frac{\kappa_i D_{i,t}}{\delta Y_i^L} - 1 \Leftrightarrow \tilde \alpha_i \ge 1 - \tilde \kappa_i
$

#### Summary
To summarize, we want to guarantee that

$\tilde \alpha_i \ge 1 - \tilde \kappa_i$

when

$
\frac{\tilde \beta_i}{d_t - \sum \left( c_i + D_{i,t} \tilde \alpha_i \right)} = u_i \ge u_i^H.
$

In [1]:
from pacti.contracts import PolyhedralIoContract
from utils import customcompose
import numpy as np

# number of components
n = 5
d_t = 1
D_t = np.ones(n)*1.0

# requirements
# A
u_H = np.ones(n)*1.0
# G
y_L = np.ones(n)*0.1

top_level = PolyhedralIoContract.from_strings(
    input_vars=[f"u_{i}" for i in range(n)],
    output_vars=[f"y_{i}" for i in range(n)],
    assumptions=[f"u_{i} >= {u_H[i]}" for i in range(n)],
    guarantees=[f"y_{i} <= {y_L[i]}" for i in range(n)]
    )


top_level_translation = PolyhedralIoContract.from_strings(
    input_vars=[f"{varname}_{i}" for varname in ["alpha_tilde", "kappa_tilde"] for i in range(n)],
    output_vars=[f"y_{i}" for i in range(n)],
    assumptions=[f"alpha_tilde_{i} >= 1 - kappa_tilde_{i}" for i in range(n)],
    guarantees=[]#[f"y_{i} <= {y_L[i]}" for i in range(n)]
    )

internal_law = PolyhedralIoContract.from_strings(
    input_vars=[f"{varname}_{i}" for varname in ["u", "beta_tilde", "c"] for i in range(n)],
    output_vars=[f"{varname}_{i}" for varname in ["alpha_tilde"] for i in range(n)],
    assumptions=#[f"u_{i} >= {u_H[i]}" for i in range(n)],
    [],
    guarantees=
    [f"beta_tilde_{i} >= {u_H[i]}*({d_t} - ({'+'.join([f'c_{j} + {D_t[j]}*alpha_tilde_{j}' for j in range(n)])}))" for i in range(n)] #+
    #[f"alpha_tilde_{i} >= 0" for i in range(n)]
    )


#internal_law.compose(top_level_translation,vars_to_keep=[f"{varname}_{i}" for varname in ["alpha_tilde"] for i in range(n)])
customcompose(top_level_translation, internal_law)



InVars: [kappa_tilde_0, kappa_tilde_1, kappa_tilde_2, kappa_tilde_3, kappa_tilde_4, u_0, u_1, u_2, u_3, u_4, beta_tilde_0, beta_tilde_1, beta_tilde_2, beta_tilde_3, beta_tilde_4, c_0, c_1, c_2, c_3, c_4]
OutVars:[y_0, y_1, y_2, y_3, y_4, alpha_tilde_0, alpha_tilde_1, alpha_tilde_2, alpha_tilde_3, alpha_tilde_4]
A: [
  
]
G: [
  alpha_tilde_1 + alpha_tilde_2 + alpha_tilde_3 + alpha_tilde_4 + beta_tilde_0 + c_0 + c_1 + c_2 + c_3 + c_4 - kappa_tilde_0 <= -2
  alpha_tilde_1 + alpha_tilde_2 + alpha_tilde_3 + alpha_tilde_4 + beta_tilde_1 + c_0 + c_1 + c_2 + c_3 + c_4 - kappa_tilde_0 <= -2
  alpha_tilde_1 + alpha_tilde_2 + alpha_tilde_3 + alpha_tilde_4 + beta_tilde_2 + c_0 + c_1 + c_2 + c_3 + c_4 - kappa_tilde_0 <= -2
  alpha_tilde_1 + alpha_tilde_2 + alpha_tilde_3 + alpha_tilde_4 + beta_tilde_3 + c_0 + c_1 + c_2 + c_3 + c_4 - kappa_tilde_0 <= -2
  alpha_tilde_1 + alpha_tilde_2 + alpha_tilde_3 + alpha_tilde_4 + beta_tilde_4 + c_0 + c_1 + c_2 + c_3 + c_4 - kappa_tilde_0 <= -2
  alpha_tilde_0 +